# Make noise to image

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def add_gaussian_noise(image, mean=0, sigma=25):
    noise = np.random.normal(mean, sigma, image.shape).astype(np.float32)
    noisy_image = cv2.add(image.astype(np.float32), noise)
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

In [2]:
image_path = "./sharp/7.jpg" 
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
gaussian_noisy = add_gaussian_noise(original_image)
cv2.imwrite("./noise/gaussian_noisy.jpg", gaussian_noisy)

True

# GA + Gaussian filter
## GA optimize the parameter of Gaussian filter

### Function definition

In [3]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error
import random

def calculate_psnr(original, blurred):
    mse = mean_squared_error(original, blurred)
    if mse == 0:
        return float("inf")  
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def create_gaussian_kernel(kernel_size, sigma):
    kernel_size = max(3, kernel_size | 1)  
    ax = np.linspace(-(kernel_size // 2), kernel_size // 2, kernel_size)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    kernel /= np.sum(kernel)  
    return kernel

def apply_custom_gaussian_filter(image, kernel_size=5, sigma=1.0):
    kernel = create_gaussian_kernel(kernel_size, sigma)
    if len(image.shape) == 3:  
        filtered_image = np.zeros_like(image)
        for i in range(3): 
            filtered_image[:, :, i] = cv2.filter2D(image[:, :, i], -1, kernel)
    else: 
        filtered_image = cv2.filter2D(image, -1, kernel)
    return filtered_image

def fitness_function(original, image, params):
    kernel_size, sigma = int(params[0]), params[1]
    kernel_size = max(3, kernel_size | 1)
    filtered_image = apply_custom_gaussian_filter(image, kernel_size, sigma)
    return calculate_psnr(original, filtered_image), kernel_size, sigma


### Only Gaussian filter

In [4]:
original = cv2.imread('./sharp/7.jpg', cv2.IMREAD_GRAYSCALE)
image = cv2.imread('./noise/gaussian_noisy.jpg', cv2.IMREAD_GRAYSCALE)
kernel_size = 7
sigma = 1.3
filtered_image = apply_custom_gaussian_filter(image, kernel_size, sigma)

psnr_value = psnr(image, filtered_image)
print(f"PSNR without GA: {psnr_value:.2f}")

cv2.imwrite('./result/only_Gaussian.jpg', filtered_image)
print("Filtered image saved as 'only_Gaussian.jpg'")

PSNR without GA: 20.57
Filtered image saved as 'only_Gaussian.jpg'


### GA to optimize the value

In [5]:
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)

original = cv2.imread('./sharp/7.jpg', cv2.IMREAD_GRAYSCALE)
image = cv2.imread('./noise/gaussian_noisy.jpg', cv2.IMREAD_GRAYSCALE)

population_size = 200
generations = 15
mutation_rate = 0.1

population = [(random.randint(3, 101) | 1, random.uniform(0.1, 3)) for _ in range(population_size)]

for gen in range(generations):
    print(f"\n--- Generation {gen + 1} ---")
    
    fitness_scores = []
    for individual in population:
        fitness, kernel, sigma = fitness_function(original, image, individual)
        fitness_scores.append(fitness)
        print(f"Tested Kernel Size: {kernel}, Sigma: {sigma:.2f}, PSNR: {fitness:.2f}")
    
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population), reverse=True)]
    population = sorted_population[:population_size // 2]
    
    new_population = []
    while len(new_population) < population_size:
        parent1, parent2 = random.sample(population, 2)
        
        child = ((parent1[0] + parent2[0]) // 2, (parent1[1] + parent2[1]) / 2)
        
        if random.random() < mutation_rate:
            child = (child[0] + random.choice([-2, 2]), child[1] + random.uniform(-0.5, 0.5))
        
        child = (max(3, int(child[0]) | 1), max(0.1, child[1]))
        
        new_population.append(child)
    
    population = new_population
    best_fitness = max(fitness_scores)
    print(f"Best PSNR in Generation {gen + 1}: {best_fitness:.2f}")

best_params = sorted_population[0]
kernel_size_ga, sigma_ga = int(best_params[0]), best_params[1]
print(f"\nOptimized Parameters: Kernel Size = {kernel_size_ga}, Sigma = {sigma_ga:.2f}")

filtered_image_ga = apply_custom_gaussian_filter(image, kernel_size_ga, sigma_ga)
psnr_value_ga = calculate_psnr(original, filtered_image_ga)
print(f"PSNR with GA: {psnr_value_ga:.2f}")

cv2.imwrite('./result/GA_Gaussian.jpg', filtered_image_ga)
print("Filtered image saved as 'GA_Gaussian.jpg'")


--- Generation 1 ---
Tested Kernel Size: 85, Sigma: 0.42, PSNR: 22.17
Tested Kernel Size: 97, Sigma: 0.90, PSNR: 28.30
Tested Kernel Size: 31, Sigma: 0.50, PSNR: 24.09
Tested Kernel Size: 17, Sigma: 2.06, PSNR: 29.16
Tested Kernel Size: 73, Sigma: 0.35, PSNR: 20.85
Tested Kernel Size: 57, Sigma: 0.19, PSNR: 20.26
Tested Kernel Size: 15, Sigma: 0.73, PSNR: 27.39
Tested Kernel Size: 67, Sigma: 1.85, PSNR: 29.24
Tested Kernel Size: 75, Sigma: 0.68, PSNR: 26.88
Tested Kernel Size: 87, Sigma: 2.13, PSNR: 29.13
Tested Kernel Size: 57, Sigma: 0.74, PSNR: 27.43
Tested Kernel Size: 79, Sigma: 0.91, PSNR: 28.33
Tested Kernel Size: 3, Sigma: 2.30, PSNR: 27.71
Tested Kernel Size: 23, Sigma: 2.12, PSNR: 29.14
Tested Kernel Size: 47, Sigma: 0.91, PSNR: 28.33
Tested Kernel Size: 31, Sigma: 2.88, PSNR: 28.72
Tested Kernel Size: 47, Sigma: 0.40, PSNR: 21.60
Tested Kernel Size: 51, Sigma: 0.38, PSNR: 21.30
Tested Kernel Size: 47, Sigma: 1.85, PSNR: 29.23
Tested Kernel Size: 9, Sigma: 2.22, PSNR: 29.14


# Comparison

In [6]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from scipy.signal import correlate2d
from skimage.filters import sobel

def calculate_ssim(original, blurred):
    return ssim(original, blurred, data_range=blurred.max() - blurred.min())

def calculate_mse(original, blurred):
    return mean_squared_error(original, blurred)

def calculate_ncc(original, blurred):
    original_normalized = original - np.mean(original)
    blurred_normalized = blurred - np.mean(blurred)
    
    numerator = np.sum(original_normalized * blurred_normalized)
    denominator = np.sqrt(np.sum(original_normalized**2) * np.sum(blurred_normalized**2))
    
    if denominator == 0:
        return 0
        
    return numerator / denominator

def calculate_gmsd(original, blurred):
    gradient_original = sobel(original)
    gradient_blurred = sobel(blurred)
    gmsd = np.sqrt(np.mean((gradient_original - gradient_blurred) ** 2))
    return gmsd

def calculate_vif(original, blurred):
    if np.array_equal(original, blurred):
        return 1.0
        
    gradient_original = np.gradient(original.astype(float))
    gradient_blurred = np.gradient(blurred.astype(float))
    sigma_nsq = 2  
    
    vif_score = 0
    total_variance = 0
    for grad_o, grad_b in zip(gradient_original, gradient_blurred):
        numerator = np.var(grad_o)
        denominator = np.var(grad_o - grad_b) + sigma_nsq
        vif_score += np.sum(numerator / denominator)
        total_variance += np.sum(numerator)  
    
    return vif_score / (total_variance + 1e-10)  

def calculate_fsim(original, blurred):
    pc1 = sobel(original).astype(np.float32)
    pc2 = sobel(blurred).astype(np.float32)
    t1 = np.minimum(pc1, pc2)
    t2 = np.maximum(pc1, pc2)
    fsim = np.sum(t1) / np.sum(t2)
    return fsim

# noisy vs original

In [7]:
original_img = cv2.imread('./sharp/7.jpg', cv2.IMREAD_GRAYSCALE)
blurred_img = cv2.imread('./noise/gaussian_noisy.jpg', cv2.IMREAD_GRAYSCALE)

print("PSNR:", calculate_psnr(original_img, blurred_img))
print("SSIM:", calculate_ssim(original_img, blurred_img))
print("MSE:", calculate_mse(original_img, blurred_img))
print("NCC:", calculate_ncc(original_img, blurred_img))
print("GMSD:", calculate_gmsd(original_img, blurred_img))
print("VIF:", calculate_vif(original_img, blurred_img))
print("FSIM:", calculate_fsim(original_img, blurred_img))


PSNR: 20.255916492349108
SSIM: 0.20471616088149933
MSE: 613.0398575498575
NCC: 0.9130192558516156
GMSD: 0.0757011837824687
VIF: 0.0032201325385556503
FSIM: 0.21155773


# only Gaussian vs original

In [8]:
original_img = cv2.imread('./sharp/7.jpg', cv2.IMREAD_GRAYSCALE)
blurred_img = cv2.imread('./result/only_Gaussian.jpg', cv2.IMREAD_GRAYSCALE)

print("PSNR:", calculate_psnr(original_img, blurred_img))
print("SSIM:", calculate_ssim(original_img, blurred_img))
print("MSE:", calculate_mse(original_img, blurred_img))
print("NCC:", calculate_ncc(original_img, blurred_img))
print("GMSD:", calculate_gmsd(original_img, blurred_img))
print("VIF:", calculate_vif(original_img, blurred_img))
print("FSIM:", calculate_fsim(original_img, blurred_img))

PSNR: 29.081201197892504
SSIM: 0.690407106918159
MSE: 80.34525641025641
NCC: 0.9871463312078532
GMSD: 0.027442218963702653
VIF: 0.02768816985579704
FSIM: 0.3913554


# GA+Gaussian vs original

In [9]:
original_img = cv2.imread('./sharp/7.jpg', cv2.IMREAD_GRAYSCALE)
blurred_img = cv2.imread('./result/GA_Gaussian.jpg', cv2.IMREAD_GRAYSCALE)

print("PSNR:", calculate_psnr(original_img, blurred_img))
print("SSIM:", calculate_ssim(original_img, blurred_img))
print("MSE:", calculate_mse(original_img, blurred_img))
print("NCC:", calculate_ncc(original_img, blurred_img))
print("GMSD:", calculate_gmsd(original_img, blurred_img))
print("VIF:", calculate_vif(original_img, blurred_img))
print("FSIM:", calculate_fsim(original_img, blurred_img))

PSNR: 29.24567218588478
SSIM: 0.7553739690765908
MSE: 77.35940883190884
NCC: 0.9876686828129491
GMSD: 0.027652224572130648
VIF: 0.028053607946252404
FSIM: 0.40505475


# GA+Gaussian vs only Gaussian

In [10]:
original_img = cv2.imread('./result/only_Gaussian.jpg', cv2.IMREAD_GRAYSCALE)
blurred_img = cv2.imread('./result/GA_Gaussian.jpg', cv2.IMREAD_GRAYSCALE)

print("PSNR:", calculate_psnr(original_img, blurred_img))
print("SSIM:", calculate_ssim(original_img, blurred_img))
print("MSE:", calculate_mse(original_img, blurred_img))
print("NCC:", calculate_ncc(original_img, blurred_img))
print("GMSD:", calculate_gmsd(original_img, blurred_img))
print("VIF:", calculate_vif(original_img, blurred_img))
print("FSIM:", calculate_fsim(original_img, blurred_img))

PSNR: 41.75876175170991
SSIM: 0.9651512669627264
MSE: 4.337147435897436
NCC: 0.9992892124475374
GMSD: 0.007904686183816446
VIF: 0.2620876691401253
FSIM: 0.7218432
